In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
List_Url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_Url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [3]:
# Ignore cells with a borough that is Not assigned.
df=df[df['Borough']!='Not assigned']

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',  df['Borough'], df['Neighborhood'])

# Group Neighbourhoods with the same Postcode
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))
df = df.reset_index()

In [4]:
# Use the Geocoder package or the csv file to create the following dataframe:
url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df = df.join(df_geo.set_index('Postal Code'), on='PostalCode')

In [5]:
df.head(12)

PostalCode      Borough                                       Neighborhood  \
0         M1B  Scarborough                                     Malvern, Rouge   
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek   
2         M1E  Scarborough                  Guildwood, Morningside, West Hill   
3         M1G  Scarborough                                             Woburn   
4         M1H  Scarborough                                          Cedarbrae   
5         M1J  Scarborough                                Scarborough Village   
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park   
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge   
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West   
9         M1N  Scarborough                        Birch Cliff, Cliffside West   
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...   
11        M1R  Scarborough                                  Wexford, Maryvale   

     Latitude  Longitude  
0   43.806686 -79.194353  
1   43.784535 -79.160497  
2   43.763573 -79.188711  
3   43.770992 -79.216917  
4   43.773136 -79.239476  
5   43.744734 -79.239476  
6   43.727929 -79.262029  
7   43.711112 -79.284577  
8   43.716316 -79.239476  
9   43.692657 -79.264848  
10  43.757410 -79.273304  
11  43.750072 -79.295849